In [1]:
using DrWatson
@quickactivate "Attitude Control"

In [2]:
using CBinding

In [3]:
LD_PATH = [
    "../build/",
]

INC_PATH = [
    "../lib/libutil/include/",
    "../lib/libadcs/include/",
    "../lib/libadcs/lib/libadcs_client/include",
]

LD_PATH = map(p -> "-L"*p, LD_PATH)
INC_PATH = map(i -> "-I"*i, INC_PATH)
LIB = "shadcs";

In [4]:
## C environment
c`$(INC_PATH) $(LD_PATH) -l$(LIB)`

In [5]:
# Define stdint sizes
const c"int8_t"  = Int8
const c"uint8_t"  = UInt8

const c"int16_t"  = Int16
const c"uint16_t"  = UInt16

const c"int32_t"  = Int32
const c"uint32_t" = UInt32

const c"int64_t"  = Int64
const c"uint64_t" = UInt64;

In [6]:
# Setup includes
c"""
#include <gs/util/types.h>
#include <gs/adcs/adcs_types.h>

#include <gs/adcs/param/host/sensor_css.h>
#include <gs/adcs/param/host/sensor_common.h>
#include <gs/adcs/param/host/sensor_fss.h>
#include <gs/adcs/param/host/sensor_extgyro.h>
#include <gs/adcs/param/host/sensor_horizon.h>
#include <gs/adcs/param/host/sensor_extmag.h>
#include <gs/adcs/param/host/sensor_startracker.h>
#include <gs/adcs/param/host/sensor_a3200.h>

#include <gs/adcs/param/host/act_magnetorquer.h>
#include <gs/adcs/param/host/act_rw.h>

#include <gs/adcs/param/host/gnc_common.h>
#include <gs/adcs/param/host/gnc_ads.h>
#include <gs/adcs/param/host/gnc_ctrl_pointing.h>
#include <gs/adcs/param/host/gnc_ctrl_spin.h>
"""

c"""
#include <gs/adcs/adcs_param_types.h>
#include <gs/adcs/adcs_telem_types.h>

#include <gs/adcs/acs/controller/gs_adcs_wheel.h>
""";

In [7]:
gnc = Libc.malloc(c"gs_adcs_config_gnc_t"(
        common=(
            inertia=(1,2,3),
        )
    )
)

periph = Libc.malloc(c"gs_adcs_config_periph_t"())
fss = Libc.malloc(c"gs_adcs_sensor_fss_memory_t"())

conf = Libc.malloc(c"gs_adcs_config_t"(
        gnc=gnc, periph=periph
    )
)

ephem = Libc.malloc(c"gs_adcs_ephem_t"())

state_est = Libc.malloc(c"GS_ADCS_UKF_Data_t"(
        EstimatedState=(
            1.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01
        )
    )
)

act_data = Libc.malloc(c"gs_adcs_actuatordata_t"())
ctrl = Libc.malloc(c"gs_adcs_ctrl_t"(ref_q=(q4=1.0,)))

# Get handle to function. Could also do c"gs_adcs_wheel_ctrl"( c"gs_adcs_wheel_ctrl")
smc = c"gs_adcs_wheel_ctrl"[] 

Cptr{Cfunction{Int8, Tuple{Cptr{Cconst{var"c\"gs_adcs_config_t\""}}, Cptr{Cconst{var"c\"gs_adcs_ephem_t\""}}, Cptr{Cconst{var"c\"GS_ADCS_UKF_Data_t\""}}, Cconst{Cptr{Cconst{var"c\"gs_adcs_actuatordata_t\""}}}, Cptr{var"c\"gs_adcs_ctrl_t\""}}, :cdecl}}(0x00007fc504114350)

In [8]:
smc(conf, ephem, state_est, act_data, ctrl)
ctrl[]

var"(c\"gs_adcs_ctrl_t\")" (180-byte struct)
  ref_q = var"(c\"quaternion_t\")" (16-byte struct)
    q1 = 0.0f0
    q2 = 0.0f0
    q3 = 0.0f0
    q4 = 1.0f0
  err_q = var"(c\"quaternion_t\")" (16-byte struct)
    q1 = 1.0f0
    q2 = 0.0f0
    q3 = 0.0f0
    q4 = 0.0f0
  ierr_q = var"(c\"quaternion_t\")" (16-byte struct)
    q1 = 0.0f0
    q2 = 0.0f0
    q3 = 0.0f0
    q4 = 0.0f0
  err_rate = Float32[0.01, 0.01, 0.01]
  M = Float32[NaN, NaN, NaN]
  mw_torque = Float32[0.0, 0.0, 0.0, 0.0]
  mw_speed = Float32[0.0, 0.0, 0.0, 0.0]
  mw_momentum = Float32[0.0, 0.0, 0.0, 0.0]
  ref_rate = Float32[0.0, 0.0, 0.0]
  euler_offset = Float32[0.0, 0.0, 0.0]
  offset_err = Float32[0.0, 0.0, 0.0]
  bodyTorque = Float32[0.0, 0.0, 0.0]
  bodyMomentum = Float32[0.0, 0.0, 0.0]